<a href="https://colab.research.google.com/github/joowop/Metaverse_AI_Project/blob/main/LDA_%EC%8A%A4%EB%85%B8%EC%9A%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install konlpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 55.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 31.9 MB/s eta 0:00:00


In [1]:
!pip install -U pyLDAvis

In [2]:
import pandas as pd

df = pd.read_csv('app.csv')
df = df.iloc[3:]
df = df.reset_index()
df = df.drop(columns=['index','Unnamed: 0'])
df.head()

,text,score
0,그동안은 잘썼으니 별두개 드립니다. 어느날인가부터 사진을 찍으면 볼 부분만 너무나도...,2
1,"안녕하세요, 잘쓰고 있긴한데 한 몇주전부터 셀카모드(고화질)로 촬영하면 렉 잠깐 걸...",5
2,무료 체험 후 구독 취소에 애먹으시고 계신 분들은 gmail로 온 결제내역 알림에 ...,3
3,별을 단 1개도 주기 아까운 어플이네요. 스노우 꽤 여러번 결제를 했구요. 처음 빼...,1
4,쓰면 쓸수록 광고가 많아지는 신기한 앱입니다. 물론 기본이 무료앱이니까 광고 보는거...,2


In [3]:
import konlpy
import re
from tqdm import tqdm

In [6]:
def tokenize_text(text):
    text = re.sub(r'[^ㄱ-ㅣ가-힣\s]','',text)

    okt = konlpy.tag.Okt()
    okt_morphs = okt.pos(text)

    words = []
    for word, pos in okt_morphs:
        if pos in ['Adjective','Verb','Noun']:
            words.append(word)

    word_str = ' '.join(words)
    return word_str

tokenize_text('안녕하세요, 잘쓰고 있긴한데 한 몇주전부터 셀카모드(고화질)로 촬영하면')

'안녕하세요 잘쓰고 있긴한데 한 주전 셀카 모드 고화질 촬영 하면'

In [7]:
token_list = []
for text in tqdm(df['text']):
    token_list.append(tokenize_text(text))


100%|██████████| 1000/1000 [00:18<00:00, 54.19it/s]


In [8]:
token_list[1]

'안녕하세요 잘쓰고 있긴한데 한 주전 셀카 모드 고화질 촬영 하면 렉 잠깐 걸렸다가 카메라 사용 할수 없다 뜨고 저장 안되네요 안되는 경우 더 많고 정말 가끔 저장 됩니다 폰 껐다 켜 봤고 스노우 어플 삭제 했다 깔아 봤습니다 폰 기종은 갤고요 후 카메라 저장 됩니다 한 이유 필터 및 고화질 셀카 찍으려 한건데 안되니 답답하네요 해결 부탁드립니다'

In [9]:
# 짧은 문장을 찾아서 corpus_list에 넣어준다. 하지만 여긴 없기 때문에 빈 리스트가 나온다.
# 짧은 문장을 찾아서 삭제
corpus_list = []

for index in range(len(token_list)):
    corpus = token_list[index]
    if len(set(corpus.split())) < 3:
        corpus_list.append(corpus)

for corpus in corpus_list:
    token_list.remove(corpus)

In [10]:
corpus_list

[]

In [11]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

In [12]:
# LDA는 count 기반의 vectorizer만 사용해야한다.'
# max_df : 전체에서 10%이상 나타나는 단어를 쓰겠다.
# min_df : 전체에서 두개 문장 미만에서 등장하는 단어를 제외한다.
# ngram_range : 단어를 만들때 두개의 단어?를 결합하여 찾아주는 함수 ex) 내 주문, 안녕 하세요. 이런식의 두개의 단어?가 얼마나 쓰였는지 찾아줌.
count_vectorizer = CountVectorizer(max_df = 0.1, min_df = 2, max_features = 1000, ngram_range = (1,2))

feat_vect = count_vectorizer.fit_transform(token_list)
feat_vect.shape

(1000, 1000)

In [13]:
# n_components : 다섯개의 비슷한 주제로 묶어봐라.
lda = LatentDirichletAllocation(n_components=5)
lda.fit(feat_vect)

LatentDirichletAllocation(n_components=5)

In [14]:
feature_names = count_vectorizer.get_feature_names_out()

def display_topics(model, feature_names, num_top_words=10):
    for topic_index, topic in enumerate(model.components_):
        print('Topic #',topic_index)
        topic_word_indexes = topic.argsort()[::-1]
        top_indexes = topic_word_indexes[:num_top_words]

        print(top_indexes)
        feature_concat = ' '.join([feature_names[i] for i in top_indexes])
        print(feature_concat)

display_topics(lda,feature_names, 20)

Topic # 0
[881 588 818 407 311 968 882 958 859 534 514 449  12 438 868 399 394 355
 262 819]
폴더 오류 찍으면 삭제 문제 핸드폰 폴더 저장 해서 카메라 폴더 앨범 안되네요 스노우 폴더 갑자기 수정 터치 사진 찍으면 사진 저장 부탁드립니다 로딩 찍은
Topic # 1
[ 19 775 675 771  53 801 594 184 755 656 653 280  11 907 579 708 248 768
 289 415]
같아요 좋은 있어서 좋아요 게임 진짜 완전 단점 조절 있는 있고 많아서 감사합니다 하는 예쁘고 장점 때문 좋습니다 메이크업 생각
Topic # 2
[ 87 875 771 801 197 907 656 168 677 311 374 662 546 947 608  46 853 263
 430 848]
광고 편집 좋아요 진짜 동영상 하는 있는 다른 있어요 문제 사람 있는데 얼굴 합니다 유저 검색 친구 리뷰 셀카 추천
Topic # 3
[839 425 691 726  28 574 422 197 331 430 433 546 168 295  33 156  84 801
 637 903]
촬영 설정 자동 적용 개선 영상 선택 동영상 버튼 셀카 소리 얼굴 다른 모드 갤럭시 누르면 광각 진짜 이후 하나
Topic # 4
[993 886 496 588 457 310 302 852  91 605 186  62 958 833 753 949 629 991
 552 355]
환불 프로필 아바타 오류 시간 문의 무료 취소 구독 유료 답변 결제 했는데 해서 처리 젤리 해결 이용 확인 없고 부탁드립니다


In [15]:
import pyLDAvis.lda_model

pyLDAvis.enable_notebook()
vis = pyLDAvis.lda_model.prepare(lda, feat_vect, count_vectorizer)
pyLDAvis.display(vis)

In [16]:
# 각 문서별 높은 확률의 토픽
sent_topic = lda.transform(feat_vect)
sent_topic[0]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


array([0.12958372, 0.04894453, 0.00585762, 0.00592337, 0.80969075])

In [17]:
sent_topic_per_list = []
for n in range(sent_topic.shape[0]):
    sent_topic_list = sent_topic[n].argmax()
    topic_per = sent_topic[n].max()
    sent_topic_per_list.append([n,sent_topic_list,topic_per])

topic_per_df = pd.DataFrame(sent_topic_per_list, columns = ['no', '토픽번호', '확률'])
topic_per_df

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,no,토픽번호,확률
0,0,4,0.809691
1,1,0,0.975956
2,2,4,0.665377
3,3,4,0.955217
4,4,2,0.464911
...,...,...,...
995,995,3,0.739632
996,996,4,0.413182
997,997,2,0.952258
998,998,1,0.552354


In [18]:
doc_topic_df = topic_per_df.join(df)
doc_topic_df

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,no,토픽번호,확률,text,score
0,0,4,0.809691,그동안은 잘썼으니 별두개 드립니다. 어느날인가부터 사진을 찍으면 볼 부분만 너무나도...,2
1,1,0,0.975956,"안녕하세요, 잘쓰고 있긴한데 한 몇주전부터 셀카모드(고화질)로 촬영하면 렉 잠깐 걸...",5
2,2,4,0.665377,무료 체험 후 구독 취소에 애먹으시고 계신 분들은 gmail로 온 결제내역 알림에 ...,3
3,3,4,0.955217,별을 단 1개도 주기 아까운 어플이네요. 스노우 꽤 여러번 결제를 했구요. 처음 빼...,1
4,4,2,0.464911,쓰면 쓸수록 광고가 많아지는 신기한 앱입니다. 물론 기본이 무료앱이니까 광고 보는거...,2
...,...,...,...,...,...
995,995,3,0.739632,"동영상 촬영시 시간제한이 없으면 참 좋겟네요, 덧 붙여 겔럭시 노트9 사용자 인데 ...",4
996,996,4,0.413182,이제껏 잘 썼기에 조용히 있었건만 더는 안되겠네요. 사진 저장경로 변경하는게 그렇게...,1
997,997,2,0.952258,스노우는요 예쁜 필터가 만이 있어서 좋아요 그냥 사진찍는거 부다 100배1000배 ...,5
998,998,1,0.552354,그냥 다 너무 좋은데 얼마 전부터 찍은 사진이 카메라에 저장되더라구요..아마도 업뎃...,5


In [19]:
doc_topic_df['토픽번호'].value_counts()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


토픽번호
2    254
3    240
4    206
1    157
0    143
Name: count, dtype: int64

In [20]:
for topic in range(len(doc_topic_df['토픽번호'].unique())):
    print('Topic No : ', topic)

    top_per_topics = doc_topic_df[doc_topic_df['토픽번호']==topic].sort_values(by = '확률', ascending = False)
    print(top_per_topics['text'].iloc[0])
    print(top_per_topics['text'].iloc[1])
    print(top_per_topics['text'].iloc[2])

Topic No :  0
제발 업데이트 전으로 돌려주세요,,정말 너무 불편하네요.. 아니 다른건 다 좋으니까 스노우 앨범 삭제 기능만 돌려주세요,,원래는 사진 찍고 스노우 앨범 들어가면 스노우 자체에서 사진 삭제할 수 있었거든요; 근데 갑자기 삭제 안되고 보정밖에 안되게 바뀌었네요;; 사진 옆으로 넘어가지도 않고 맘에 안드는 사진 삭제할려면 폰갤러리까지 가서 그 사진 다시 찾아서 삭제하고 다시 스노우와서 사진찍고 진짜 너무 불편해요. 제발 스노우 앨범에서 바로 삭제할 수 있는 기능 돌려주세요.
안녕하세요, 잘쓰고 있긴한데 한 몇주전부터 셀카모드(고화질)로 촬영하면 렉 잠깐 걸렸다가 카메라를 사용할수없다뜨고 저장이 안되네요. (안되는 경우가 더 많고, 정말 가끔 저장됩니다.) 폰껐다 켜봤고, 스노우 어플 삭제했다 깔아봤습니다. 폰기종은 갤s22고요, 후면카메라는 저장됩니다. vip한 이유가 vip필터 및 고화질로 셀카찍으려 한건데 안되니 답답하네요 ㅜ 해결 부탁드립니다..
무난하게 잘 사용하고 있었는데 오늘 갑자기 '오려내기' 기능으로 편집한 사진이 저장되지 않고 무한 로딩되며, DSLR 효과를 적용했을 때에도 효과가 적용되지 않고 무한 로딩되는 일이 발생하였습니다. 빠른 해결 부탁드립니다.
Topic No :  1
진짜좋읍니다장점.1.이펙트드러가면 귀여운이모티콘이있어서사진찍을때예쁘다.2.사진찍을때립스틱이나 볼터치가되있어서그냥사진보다더예쁘다.3.자기가 윈하는 이모티콘으로 새로운이모티콘을만들수있어서좋다(회원가입해야되지만....)4.재밌고 웃긴이모티코이랑 미니게임을 할수있다.5.여자들이 좋아하게 뷰티도만들어져있다.(예:갈색눈동자)6.어린애들이좋아하겠금만들었다.7.나쁜게없어서좋다.8.사진찍고 옆에사진들어가서메이크업,얼굴보정등등을할수있어서좋다.(쉽게말해서메이크업을사진찍고도할수있다는말이다.)9.스노우들어가면 이벤트갔은게있어서 바로찾아볼수있다. 장점 -끝- 이렇게좋은점이많으니강력추천드리고요끝까지긴글읽어주셔서감사합니다 그럼여기서마치도로하겠습니다.
아니 이거 진짜 제가 테크닉

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
